In [1]:
def min_orbits(elems,group_elems,act,key):
    assocs = {e : e for e in elems}
    def follow_through(e):
        while not assocs[e] == e:
            e = assocs[e]
        return e
    
    for e in elems:
        for g in group_elems:
            a = follow_through(e)
            b = follow_through(act(g,e))
            if key(a)<key(b):
                assocs[b]=a
            elif key(a)>key(b):
                assocs[a]=b
    result=[e for e in elems if assocs[e]==e]
    result.sort(key=key)
    return result

In [2]:
import functools

def nextDisks(R,n,disks,z):
    pi = R.uniformiser()
    newDisks = []
    for disk in disks:
        polynomialReduced = disk["polynomial"].change_ring(R.residue_field())
        #other case: polynomialReduced == 0
        if polynomialReduced.is_zero():
            newdisk = dict(center = disk["center"], polynomial = disk["polynomial"]/pi, size=disk["size"])
            newDisks.append(newdisk)
        else:     
            for root,_ in polynomialReduced.roots():
                rootlift = R(root).lift_to_precision()
                newpoly = disk["polynomial"](pi*z+rootlift)/pi
                newcenter = disk["center"]+rootlift*pi**disk["size"]
                newdisk = dict(center = newcenter, polynomial = newpoly, size=disk["size"]+1)
                newDisks.append(newdisk)
    return newDisks

def findApproximateZeros(R,f,n):
    pi = R.uniformiser()
    Rpoly.<z> = R[]
    f2 = f(z)
    disks = [dict(
        center=0,
        polynomial=f2,
        size=0,
    )]
    for precision in range(n):
        disks = nextDisks(R,precision,disks,z)
    return disks

def hasZerosUpToPrecision(R,f,nmax=None):
    pi = R.uniformiser()
    Rpoly.<z> = R[]
    f2 = f(z)
    disks = [dict(
        center=0,
        polynomial=f2,
        size=0,
    )]
    precision = 0
    while(disks != [] and (nmax == None or precision < nmax)):
        newdisks = nextDisks(R,precision,disks,z)
        disks = []
        for disk in newdisks:
            if(disk["polynomial"].change_ring(R.residue_field()).degree() == 1):
                return None #infinite precision
            else:
                disks.append(disk)
        precision += 1
    return precision - 1

def automorphism_action(W,E,n):
    Wpoly.<z> = W[]
    Ez = E(z)
    R.<pi> = W.extension(Ez)
    F = R.residue_field()
    f = F.degree()
    transforms = []
    for i in range(f):
        poly = Ez.map_coefficients(W.frobenius_endomorphism(i))
        disks = findApproximateZeros(R,poly,n)
        for disk in disks:
            if i==0 and disk["center"].is_equal_to(pi,disk["size"]):
                x = F.primitive_element()
                for j in range(f):
                    transforms.append((disk["polynomial"].change_ring(F)(x^j),1,i))
            else:
                #could probably take only one for each i, still generate
                transforms.append((F(disk["polynomial"](0)),F((disk["center"]/pi)^n),i))
    return transforms

def eisenstein_to_canonicalrelation(W,E,n):
    if(E.degree() >= 2):
        R.<pi> = W.extension(E)
    else:
        R = W
        pi = W.residue_field().characteristic()
    K = R.residue_field()
    p = K.characteristic()
    x = R(p)/pi
    a = []
    for i in range(n-1):
        #print(x)
        nexta = K(x)
        a.append(nexta)
        x = (x - R.teichmuller(nexta))/pi
    return a

def canonicalrelation_to_eisenstein(W,coeffs,n):
    Wpoly.<z> = W[]
    K = W.residue_field()
    p = K.characteristic()
    e=1
    while(K(coeffs[e-1]) == 0):
        e+=1
        if(e-1 >= len(coeffs)):
            raise Exception("invalid coefficients")
    if(n<=e):
        raise Exception("length smaller than ramification")
    g = p
    for i in range(len(coeffs)):
        g -= W.teichmuller(coeffs[i])*z^(i+1)
    polynomials = {}
    for i in range(n-1,e-1,-1):
        polynomial = g * z^(i-e)
        polynomial = polynomial % z^n
        for j in range(n-1,i,-1):
            if(j < len(polynomial.list())):
                coeff = polynomial.list()[j]
                polynomial -= coeff * polynomials[j]
        polynomial = polynomial / polynomial.list()[i]
        polynomials[i] = polynomial
    return polynomials[e]

def chainRingMass(W,E,n):
    if n<=E.degree():
        return 1
    R.<pi> = W.extension(E)
    p = W.residue_field().characteristic()
    f = W.residue_field().degree()
    q = p^f
    aut = 0
    for i in range(f):
        Etwist = E.map_coefficients(W.frobenius_endomorphism(i))
        for disk in findApproximateZeros(R,Etwist,n):
            aut = aut + q^(n-disk["size"])
    return 1/aut


#transform (a,l,i) means 
# [E(z)] -> phi^i(E(0))/E(0) * [E(z)] + a[z^n]
# [z^n] -> l [z^n]
# phi^i-semilinear
# So [pE(z)/E(0)] -> [pE(z)/E(0)] + a*(p/phi^i(E(0)))[z^n]
# and finally, [g(z)] = [pE(z)/E(0)] + a_n [z^n] where g(z) = p - sum [a_i]z^i
# So [g(z)] -> [g(z)] + (a*(p/phi^i(E(0))) + phi^i(a_n) l - a_n)[z^n]
# On invariants wrt [g(z)],[z^n] basis, take
# b -> phi^{-i}((b + (a*(p/phi^i(E(0))) + phi^i(a_n) l - a_n)) / l)
def compute_branches(W,E,n):
    K = W.residue_field()
    p = K.characteristic()
    e = E.degree()
    Wpoly.<z> = W[]
    Ez = E(z)    
    _,_,iso2 = K.free_module(map=True)
    def compare(a,b):
        for i in range(len(iso2(a))-1,-1,-1):
            if iso2(a)[i]<iso2(b)[i]:
                return -1
            elif iso2(a)[i]>iso2(b)[i]:
                return 1
        return 0
    transforms = automorphism_action(W,E,n)
    an = eisenstein_to_canonicalrelation(W,E,n+1)[n-1]
    maps = []
    def act(transform,x):
        a,l,i=transform
        return (K.frobenius_endomorphism(-i)((x + a*K.frobenius_endomorphism(i)(K(p/E(0))) + K.frobenius_endomorphism(i)(an) * l - an) / l))      
    branch_eisensteins=[]
    for a in min_orbits(K,transforms,act,functools.cmp_to_key(compare)):
        #print(a)
        branch_eisensteins.append(Ez - (Ez(0)/p)*(-Ez(0))^(n//e)*W(a-an).lift_to_precision()*z^(n%e))
    return branch_eisensteins

def compute_init(W,e):
    K = W.residue_field()
    f = K.degree()
    Wpoly.<z> = W[]
    p = K.characteristic()
    _,_,iso2 = K.free_module(map=True)
    def compare(a,b):
        for i in range(len(iso2(a))-1,-1,-1):
            if iso2(a)[i]<iso2(b)[i]:
                return -1
            elif iso2(a)[i]>iso2(b)[i]:
                return 1
        return 0
    elems = [e for e in K if not e==0]
    group_elems = [(e,i) for e in elems for i in range(f)]
    def act(pair,y):
        x,i = pair
        return K.frobenius_endomorphism(i)(y)*x^e
    init_eisensteins = []
    for a in min_orbits(elems,group_elems,act,functools.cmp_to_key(compare)):
        init_eisensteins.append(z^e - p/W(a).lift_to_precision())
    return init_eisensteins
        
    

In [3]:
W.<x> = Zq(4)
compute_init(W,6)

[(1 + O(2^20))*z^6 + 2 + 2^2 + 2^3 + 2^4 + 2^5 + 2^6 + 2^7 + 2^8 + 2^9 + 2^10 + 2^11 + 2^12 + 2^13 + 2^14 + 2^15 + 2^16 + 2^17 + 2^18 + 2^19 + 2^20 + O(2^21),
 (1 + O(2^20))*z^6 + (x + 1)*2 + O(2^21)]

In [4]:
def compute_tree(W,e,tag_lmfdb=True):
    root = {}
    root["children"] = [compute_tree_rec(W,E,e+1,root) for E in compute_init(W,e)]
    tag_tree_nice_eisenstein(W,root)
    if(tag_lmfdb):
        tag_tree_lmfdb(root,W,e)
    return root

def compute_tree_rec(W,E,n,parent):
    node = {}
    node["length"] = n
    node["eisenstein"] = E
    node["mass"] = chainRingMass(W,E,n)
    node["canonical_sequence"] = eisenstein_to_canonicalrelation(W,E,n)
    if("discriminant_exp" in parent and 2*parent["discriminant_exp"]+1<n): #todo check!
        node["children"] = []
        node["stable"] = True
        return node
    if("ramification" in parent):
        node["ramification"] = parent["ramification"]
    else:
        node["ramification"] = n-1 #this only happens in first step
    if("discriminant_exp" in parent):
        node["discriminant_exp"] = parent["discriminant_exp"]
    else:
        R.<pi> = W.extension(E)
        d = E.derivative()(pi).valuation()
        if d<n:
            node["discriminant_exp"] = d
    node["children"] = [compute_tree_rec(W,E2,n+1,node) for E2 in compute_branches(W,E,n)]
    if(len(node["children"])==1 and "stable" in node["children"][0] and node["children"][0]["stable"]):
        node["children"] = []
        node["stable"] = True
    return node

def min_rep_mod(n,m):
    result = n % m
    if result > m // 2:
        result -= m
    return result

def nice_eisenstein(W,E,n):
    p = W.residue_field().characteristic()
    R1.<x> = ZZ[]
    R2.<z> = R1[]
    e = E.degree()
    newcoeffs = []
    for i in range(e):
        modulus = p^(- ((i-n)//e))
        newcoeffs.append(sum([x^j*min_rep_mod(ZZ(a),modulus) for j,a in enumerate(E.list()[i].polynomial().list())]))
    newcoeffs.append(R1(1))
    result = sum([z^i*coeff for i,coeff in enumerate(newcoeffs)])
    return result
            

def tag_tree_nice_eisenstein(W,node):
    if("eisenstein" in node):
        node["nice_eisenstein"] = nice_eisenstein(W,node["eisenstein"],node["length"])
    for child in node["children"]:
        tag_tree_nice_eisenstein(W,child)
        
from urllib import request
import json

def identify_extension(node,W,poly):
    res = identify_extension_rec(node,W,poly)
    return res
    
def identify_extension_rec(node,W,poly):
    if(node["children"] == []):
        R.<pi> = W.extension(node["eisenstein"])
        prec = hasZerosUpToPrecision(R,poly)
        if(prec == None):
            return node
        else:
            return None
    for child in node["children"]:
        leaf = identify_extension_rec(child,W,poly)
        if not leaf == None:
            return leaf
    return None

#def identify_extension(node,W,poly):
#    res,_ = identify_extension_rec(node,W,poly)
#    return res

#this only works if poly is Eisenstein!
#def identify_extension_rec(node,W,poly):
#    #compare with tree recursively.
#    #either E agrees with the entire tree,
#    #but up to a level of agreement smaller than the depth,
#    #in which case we return that level of agreement and backtrack,
#    #or it agrees with all children up to tree["depth"], and with one better.
#    if(node["children"] == []):
#        R.<pi> = W.extension(node["eisenstein"])
#        prec = hasZerosUpToPrecision(R,poly)
#        return node,prec
#    for child in node["children"]:
#        print(f"child:{child},poly:{poly}")
#        leaf, prec = identify_extension_rec(child,W,poly)
#        if prec == None or ("length" in node and prec < node["length"]):
#            return leaf, prec
#    print("I guess we should not end up here!")
#    print(f"node: {node}")
#    print(f"polynomial: {poly}")

def count_leaves(node):
    if(len(node["children"]) == 0):
        return 1
    result = 0
    for child in node["children"]:
        result += count_leaves(child)
    return result

def tag_tree_lmfdb(node,W,e):

    
    p = W.residue_field().characteristic()
    f = W.residue_field().degree()
    Wpoly.<z> = W[]
    query = f"https://www.lmfdb.org/padicField/?Submit=sage&download=1&query={{%27p%27:%20{p},%20%27f%27:%20{f},%20%27e%27:%20{e}}}"
    datastr = request.urlopen(query).read().decode('utf8')

    _,rest1 = datastr.split("labels  =  ")
    labelstr,rest2 = rest1.split("data  =  ")
    datastr,_ = rest2.split("def")

    #it's not actually json, but python, but since it's just a list this also works
    labels = json.loads(labelstr)
    data = json.loads(datastr)

    if(len(labels) != len(data)):
        raise Exception("lmfdb data mismatched")
    leaves = count_leaves(node)
    unmatched = 0
    lmfdb_entries = len(labels)
    
    for i in range(len(labels)):
        poly = 0
        for j in range(len(data[i][1])-1,-1,-1):
            poly = z*poly + W(data[i][1][j])      
        leaf = identify_extension(node,W,poly)
        if not leaf==None:
            leaf["lmfdb"] = labels[i]
        else:
            unmatched+=1
            print(f"UNMATCHED: lmfdb {labels[i]}, poly {poly}")
    if(unmatched > 0):
        print(f"There have been matching errors. {unmatched} of {lmfdb_entries} lmfdb entries not matched, we have {leaves} leaves.")
        

In [1]:
def generateTooltip(node):
    if("eisenstein" in node):
        return str(node["eisenstein"])
    else:
        return ""

def generateLabel(node):
    return ""

class DiagramGenerator:
    def __init__(self,tree):
        self.style = ''
        self.body = ''
        self.script_def = ''
        self.script_redraw = ''
        self.free_id = 0
        self.tree = tree
    
    def set_style(self):
        self.style += 'body{margin: 0;}'
        self.style += '.node{margin: auto; padding: 3px 5px 3px 5px; border-radius: 5px; border: 1px solid black;}'
        self.style += '.leaf{margin: auto auto auto 0;}'
        self.style += '.tree{display: grid; row-gap: 10px; column-gap: 10px;}'
        self.style += '.tree>svg{overflow:visible; position:absolute; top:0; left: 0; height: 100%; width: 100%; z-index: -1;}'
        self.style += 'line{stroke: black;}'
    
    def set_script(self):
        self.script_def += 'function anchorEast(elem_id){'
        self.script_def += 'console.log(elem_id);'
        self.script_def += 'let elem = document.getElementById(elem_id);'
        self.script_def += 'let x = elem.offsetLeft + elem.clientWidth;'
        self.script_def += 'let y = elem.offsetTop + elem.clientHeight/2;'
        self.script_def += 'return {x:x,y:y};'
        self.script_def += '}'
        
        self.script_def += 'function anchorWest(elem_id){'
        self.script_def += 'let elem = document.getElementById(elem_id);'
        self.script_def += 'let x = elem.offsetLeft;'
        self.script_def += 'let y = elem.offsetTop + elem.clientHeight/2;'
        self.script_def += 'return {x:x,y:y};'
        self.script_def += '}'
 
        self.script_def += 'function drawEdge(sx,sy,tx,ty){'
        self.script_def += 'let svg = document.getElementById("svg");'
        self.script_def += 'let line = document.createElementNS("http://www.w3.org/2000/svg", "line");'
        self.script_def += 'line.setAttribute("x1",sx);'
        self.script_def += 'line.setAttribute("x2",tx);'
        self.script_def += 'line.setAttribute("y1",sy);'
        self.script_def += 'line.setAttribute("y2",ty);'
        self.script_def += 'svg.appendChild(line);'
        self.script_def += '}'
        
        self.script_def += 'function clearSvg(){'
        self.script_def += 'let svg = document.getElementById("svg");'
        self.script_def += 'while(svg.firstChild){'
        self.script_def += 'svg.removeChild(svg.firstChild);'
        self.script_def += '}'
        self.script_def += '}'
        
        self.script_def += 'function drawEdges(parent_id,child_ids){'
        self.script_def += 'console.log(parent_id);'
        self.script_def += 'let parentPos=anchorEast(parent_id);'
        self.script_def += 'let childPos=[];'
        self.script_def += 'child_ids.forEach((c) => childPos.push(anchorWest(c)));'
        self.script_def += 'let min_x = childPos[0].x;'
        self.script_def += 'let min_y = childPos[0].y;'
        self.script_def += 'let max_y = childPos[0].y;'
        self.script_def += 'childPos.forEach((c) => {'
        self.script_def += 'if(c.x < min_x){'
        self.script_def += 'min_x=c.x;'
        self.script_def += '}'
        self.script_def += 'if(c.y < min_y){'
        self.script_def += 'min_y=c.y;'
        self.script_def += '}'
        self.script_def += 'if(c.y > max_y){'
        self.script_def += 'max_y=c.y;'
        self.script_def += '}'
        self.script_def += '});'
        self.script_def += 'drawEdge(parentPos.x,parentPos.y,(min_x+parentPos.x)/2,parentPos.y);'
        self.script_def += 'drawEdge((min_x+parentPos.x)/2,min_y,(min_x+parentPos.x)/2,max_y);'
        self.script_def += 'childPos.forEach((c) => drawEdge((min_x+parentPos.x)/2,c.y,c.x,c.y));'
        self.script_def += '}'
        
        self.script_def += 'function redraw(){'
        self.script_def += 'clearSvg();'
        self.script_def += self.script_redraw
        self.script_def += '}'
        
    
    def get_id(self):
        new_id = "node"+str(self.free_id)
        print(new_id)
        self.free_id += 1
        return new_id
    
    #returns node id
    def add_node(self,label,row_begin,row_span,column_begin,column_span,nodeclass):
        node_id = self.get_id()
        self.body += f'<div id="{node_id}" class="{nodeclass}" style="grid-row: {row_begin} / span {row_span}; grid-column: {column_begin} / span {column_span};">'
        self.body += label
        self.body += '</div>'
        return node_id
    
    def add_edges(self,parent_id,child_ids):
        child_id_str = '['
        first=True
        for c in child_ids:
            if(not first):
                child_id_str += ','
            child_id_str += f'"{c}"'
            first = False
        child_id_str += ']'
        self.script_redraw += f'drawEdges("{parent_id}",{child_id_str});'    
    
    #returns node id of root and height
    def add_tree_rec(self,node,row_begin,column_begin):
        if (not "children" in node) or node["children"] == []:
            mass = ""
            if("mass" in node):
                mass = str(node["mass"])
            eisenstein = ""
            if("nice_eisenstein" in node):
                eisenstein = f'\\({latex(node["nice_eisenstein"])}\\)'
            node_id = self.add_node(mass,row_begin,1,column_begin,1,"node")
            self.add_node(eisenstein,row_begin,1,column_begin+1,1,"leaf")
            return node_id,1
        height = 0
        child_ids = []
        for child in node["children"]:
            child_id,child_height = self.add_tree_rec(child,row_begin+height,column_begin+1)
            child_ids.append(child_id)
            height += child_height
        label = ""
        if("mass" in node):
            label = str(node["mass"])
        node_id = self.add_node(label, row_begin,height,column_begin,1,"node")
        self.add_edges(node_id,child_ids)
        return node_id,height
    
    def bake(self):
        self.set_style()
        self.add_tree_rec(self.tree,1,1)
        self.set_script()
        html = '<!doctype html><head>'
        html += '<script type="text/javascript" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-chtml.js">'
        html += '</script>'
        html += '<style>'
        html += self.style
        html += '</style>'
        html += '<script>'
        html += 'window.MathJax = {'
        html += 'startup: {'
        html += 'ready: () => {'
        html += 'MathJax.startup.defaultReady();'
        html += 'MathJax.startup.promise.then(() => {'
        html += 'redraw();'
        html += '});'
        html += '}'
        html += '}'
        html += '};'
        html += self.script_def
        html += 'document.addEventListener("DOMContentLoaded", function () {'
        html += 'redraw()'
        html += '});'
        html += 'window.addEventListener("resize", function () {'
        html += 'redraw()'
        html += '});'
        html += '</script>'
        html += '</head>'
        html += '<body>'
        html += '<div class="tree">'
        html += self.body
        html += '<svg id="svg">'
        html += '</svg>'
        html += '</div>'
        html += '</body>'
        return html
        
def generateTreeDiagram(node):
    generator = DiagramGenerator(node)
    return generator.bake()
    
def compute_label(node):
    label = ""
    if("mass" in node):
        label += '<span class="label">'
        label += str(node["mass"])
        tooltips = []
        if("nice_eisenstein" in node):
            tooltips.append(f'<div>\\({latex(node["nice_eisenstein"])}\\)</div>')
        if("canonical_sequence" in node):
            tooltips.append(f'<div>{str(node["canonical_sequence"])}</div>')
        if("length" in node):
            tooltips.append(f'<div>length {str(node["length"])}</div>')
        if("discriminant_exp" in node):
            tooltips.append(f'<div>disc.exp. {str(node["discriminant_exp"])}</div>')
        if(len(tooltips)>0):
            label += '<div class="tooltiptext">'
            for tooltip in tooltips:
                label += tooltip
            label += '</div>'
        if(len(node["children"]) == 0 and "lmfdb" in node):
            label += '<div class="sidelabel">'
            tag = node["lmfdb"]
            url = f"https://www.lmfdb.org/padicField/{tag}"
            label += f'<a href={url}>{tag}</a>'
            label += '</div>'
        label += '</span>'
    return label

def generateTreeData(node):
    html = ''
    html += '<ul>'
    html += compute_label(node)
    if "children" in node:
        for child in node["children"]:
            html += '<li>'
            html += generateTreeData(child)
            html += '</li>'
    html += '</ul>'
    return html

#with open("test.html", 'w') as html:
#        html.write(generateTreeDiagram(compute_tree(W,e)))


def generateHTML(q,e,tag_lmfdb=True):
    print(f"Generating HTML for q={q} and e={e}")
    W.<x> = Zq(q,prec=40)
    tree = generateTreeData(compute_tree(W,e,tag_lmfdb))
    with open("html/template.html", 'r') as template, open(f"html/tree_{q}_{e}.html",'w') as output:
        template_str = template.read()
        output_str = template_str.format(tree=tree)
        output.write(output_str)

import re


def generateIndex():
    pairs = {}
    qs = set()
    es = set()
    for file in os.scandir("html"):
        #print(file.name)
        x = re.search(r"tree_(\d*)_(\d*)\.html",file.name)
        #print(x)
        if x:
            q = int(x.group(1))
            e = int(x.group(2))
        else:
            continue
        if not q in qs:
            qs.add(q)
        if not e in es:
            es.add(e)
        pairs[q,e] = file.name
    tablecontent = '<tr>'
    tablecontent += '<th>e\\q</th>'
    for q in sorted(qs):
        tablecontent += f'<th>{q}</th>'
    tablecontent += '</tr>'
    for e in sorted(es):
        tablecontent += '<tr>'
        tablecontent += f'<th>{e}</th>'
        for q in qs:
            link = ''
            if (q,e) in pairs:
                link = f'<a href="{pairs[q,e]}">*</a>'
            tablecontent += f'<td>{link}</td>'
        tablecontent += '</tr>'
    with open("html/index-template.html", 'r') as template, open(f"html/index.html",'w') as output:
        template_str = template.read()
        output_str = template_str.format(tablecontent=tablecontent)
        output.write(output_str)

In [2]:
generateIndex()

In [14]:
e_limits = {}
e_limits[2]=7
e_limits[3]=8
e_limits[4]=7
e_limits[5]=10
e_limits[7]=10
e_limits[8]=3
e_limits[9]=8
e_limits[16]=3
e_limits[25]=9
e_limits[27]=5
e_limits[32]=3



for q in [9,16,25,27,32]:
    for e in [2..e_limits[q]]:
        generateHTML(q,e,True)
generateIndex()

Generating HTML for q=9 and e=2
Generating HTML for q=9 and e=3
Generating HTML for q=9 and e=4
Generating HTML for q=9 and e=5
Generating HTML for q=9 and e=6
Generating HTML for q=9 and e=7
Generating HTML for q=9 and e=8
Generating HTML for q=16 and e=2
Generating HTML for q=16 and e=3
Generating HTML for q=25 and e=2
Generating HTML for q=25 and e=3
Generating HTML for q=25 and e=4
Generating HTML for q=25 and e=5
Generating HTML for q=25 and e=6
Generating HTML for q=25 and e=7
Generating HTML for q=25 and e=8
Generating HTML for q=25 and e=9
Generating HTML for q=27 and e=2
Generating HTML for q=27 and e=3
Generating HTML for q=27 and e=4
Generating HTML for q=27 and e=5
Generating HTML for q=32 and e=2
Generating HTML for q=32 and e=3


In [354]:
def hensel_factor(f,d,prec):
    W=f.parent().base_ring()
    Wpoly = f.parent()
    q,r = f.quo_rem(d)
    g0,a0,b0 = xgcd(d.change_ring(W.residue_field()),q.change_ring(W.residue_field()))
    a = Wpoly(a0).map_coefficients(lambda x : x.lift_to_precision())
    b = Wpoly(b0).map_coefficients(lambda x : x.lift_to_precision())
    if(not r.change_ring(W.residue_field()) == 0):
        raise Exception(f"d not dividing f mod higher valuation, r = {r}")
    if(g0 == 0 or g0.degree()>0):
        raise Exception(f"d, f/d not coprime, gcd = {g}")
    #print(f"check: b={a}")
    while(any([c.valuation() < prec for c in r.coefficients()])):
        #print(f"d = {d}")
        _,correction = (r*b).quo_rem(d)
        #print(f"corr = {correction}")
        d = d + correction
        q,r = f.quo_rem(d)
        #print(f"r = {r}")
    return d

In [486]:
#playing around

W.<x> = Zq(4,prec=40)
K = W.residue_field()
x0 = K(x)
Wpoly.<z> = W[]
#2.12.16.20 offending
f = z^12 + 4*z^11 + 4*z^10 + 4*z^9 + 12*z^8 + 8*z^7 + 2*z^6 + 4*z^5 + 4*z^4 -4*z^3 -4*z^2 + 4

#should also be presented by:
g = z^6 + 2*z^5 + 2*z^3 + 2*z^2 + 2*x + 2
#(frobenius twist of relative Eisenstein from lmfdb)

R.<pi> = W.extension(z^6 +2*z^3 + 2*x+2)

#R=R2
R2.<pi2> = W.extension(g)

R3.<pi3> = W.extension(z^6 +2*z^3 + 2*x-2)

if(hasZerosUpToPrecision(R,g)==None):
    print("g match!")

if(hasZerosUpToPrecision(R,f)==None):
    print("f match!")
else:
    print(hasZerosUpToPrecision(R,f))

#looks like f does not generate a field with f=2,e=6.
#It is also not reducible, since otherwise it would admit zeros in f=1,e=6 extensions and hence also in f=2,e=6 extensions.
#Since its Newton slope is 1/6, the only other option is f=1, e=12. But the code below definitely checks that it factors over W(F_4)!
#So there is a bug somewhere in our code.

Wpoly2.<y> = Wpoly[]
fy = f(y)
q = z - y^6
f2=0
d = fy.degree()
for i,c in enumerate(fy.resultant(q).list()):
    f2 += z^i * c / 2^(d-i)

#zeros of f2 are alpha^6/2, so if f has a zero, f2 does too.

#this checks that mod p, f2 factors into two coprime factors over F4
#f2.change_ring(W.residue_field()).factor()

#f3 is one of the hensel factors of f2
f3 = hensel_factor(f2,(z+x)^6,100)
#doublecheck
#f2.quo_rem(f3)

#we can continue to try and turn this into an Eisenstein polynomial
f4 = f3(z-x)
#tbc, f4 has slope 1/3 now, so we would need to pass to the polynomial with zeros alpha^3/2, shift,
#and at some point reach either a slope of 1/6 or a hensel factor

f4y = f4(y)
q = z - y^3
f5 = 0
d = f4y.degree()
for i,c in enumerate(f4y.resultant(q).list()):
    f5 += z^i * c / 2^(d-i)


#this is *still* wrong, but every degree 0 polynomial should have a zero somewhere
print(hasZerosUpToPrecision(R2,f4))

#confirmed below, this degree 6 polynomial does not seem to admit a zero in any of the extensions we compute.

f5.change_ring(W.residue_field()).factor()

f6 = f5(z-x)

f6.newton_polygon()

f6.list()

g match!
18
12


[x*2 + 2^2 + x*2^4 + (x + 1)*2^5 + x*2^6 + x*2^8 + 2^9 + 2^10 + (x + 1)*2^11 + (x + 1)*2^12 + 2^13 + x*2^14 + 2^15 + (x + 1)*2^16 + 2^17 + (x + 1)*2^18 + 2^19 + 2^20 + 2^21 + 2^22 + 2^23 + O(2^24),
 x*2^2 + x*2^3 + 2^4 + x*2^5 + x*2^7 + x*2^8 + (x + 1)*2^9 + x*2^10 + (x + 1)*2^11 + 2^12 + 2^13 + x*2^14 + x*2^15 + (x + 1)*2^18 + x*2^20 + (x + 1)*2^21 + (x + 1)*2^22 + (x + 1)*2^23 + (x + 1)*2^24 + O(2^25),
 (x + 1)*2^3 + 2^4 + (x + 1)*2^6 + 2^7 + 2^8 + 2^9 + 2^10 + x*2^11 + 2^13 + x*2^14 + 2^15 + (x + 1)*2^16 + (x + 1)*2^17 + (x + 1)*2^20 + 2^21 + (x + 1)*2^22 + (x + 1)*2^23 + (x + 1)*2^24 + (x + 1)*2^25 + O(2^26),
 2 + (x + 1)*2^2 + (x + 1)*2^3 + (x + 1)*2^4 + 2^5 + (x + 1)*2^6 + 2^7 + x*2^8 + (x + 1)*2^10 + x*2^12 + (x + 1)*2^13 + x*2^14 + x*2^15 + (x + 1)*2^16 + x*2^17 + 2^21 + O(2^27),
 2 + x*2^5 + (x + 1)*2^6 + x*2^8 + (x + 1)*2^9 + 2^10 + x*2^11 + (x + 1)*2^12 + (x + 1)*2^13 + 2^14 + (x + 1)*2^15 + x*2^16 + 2^17 + (x + 1)*2^18 + x*2^19 + O(2^28),
 x*2 + 2^2 + 2^3 + (x + 1)*2^7 + x*

In [487]:
#error is in
print(compute_branches(W,z^6 +2*z^3 + 2*x+2,12))
#this yields two isomorphic ones!
nice_eisenstein(W,canonicalrelation_to_eisenstein(W,[0,0,0,0,0,x0,0,0,x0+1,0,0,1],14),14)

automorphism_action(W,z^6 +2*z^3 + 2*x+2,12)

#findApproximateZeros(R,z^6+2*z^3 + 2*x+2, 12)

#nice_eisenstein(W,(z^6 +2*z^3 + 2*x+2) - (2*(x+1))^2,14)

0
x0
[(1 + O(2^40))*z^6 + (2 + O(2^41))*z^3 + (x + 1)*2 + 2^2 + O(2^41), (1 + O(2^40))*z^6 + (2 + O(2^41))*z^3 + (x + 1)*2 + (x + 1)*2^2 + O(2^41)]


[(x0 + 1, 1, 0), (0, 1, 0), (0, 1, 0), (0, 1, 0)]

In [414]:
root=compute_tree(W,6,False)
root

KeyboardInterrupt: 

In [ ]:
findApproximateZeros()

In [482]:
#print(identify_extension(root,W,f6))
hasZerosUpToPrecision(R2,f6)

f7 = f6.map_coefficients(W.frobenius_endomorphism(1))

hasZerosUpToPrecision(R2,f7)

findApproximateZeros(R3,f7,12)[0]["polynomial"].change_ring(W.residue_field())

print(hasZerosUpToPrecision(R,z^6 +2*z^3 + 2*x+2))

compute_branches(z^6 +2*z^3 + 2*x+2,12)

TypeError: no common canonical parent for objects with parents: '2-adic Unramified Extension Ring in x defined by x^2 + x + 1' and 'Univariate Polynomial Ring in z over 2-adic Eisenstein Extension Ring in pi2 defined by z^6 + 2*z^5 + 2*z^3 + 2*z^2 + 2*x + 2 over its base ring'

In [ ]:

E=canonicalrelation_to_eisenstein(W,[0,1,1,x0],5)

nice_eisenstein(W,E,5)
#eisenstein_to_canonicalrelation(W,z^2+2*z+2+4*x,5)

In [ ]:
f((x+1)*pi)

In [ ]:
error_node


In [ ]:
eisenstein_to_canonicalrelation(W,z^3-3,4)

In [ ]:
automorphism_action(W,z^3-3,4)

In [ ]:
compute_init(W,3)

In [ ]:
W.<x>=Zq(4)
K=W.residue_field()
_,_,iso2 = K.free_module(map=True)


In [271]:
xgcd?

In [ ]:
iso2(1)

In [ ]:
W.